# Regression - Ev Fiyatları Tahmin Etme

In [4]:
import pandas as pd #pandas verileri okumak için kullanılır
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error

import warnings #uyarıları kapatmak
warnings.filterwarnings('ignore')

#dosyayı okutma
df=pd.read_csv('kc_house_data.csv')

#outlier çıkartma
df=df[df['bedrooms']<7]
df=df[df['bathrooms']<5]

#type'ını kategori yapma 
df['zipcode']=df['zipcode'].astype('category')

df['age']=2023-df['yr_built'] # yaşını hesaplıyacağız
df['bedrooms']=df['bedrooms']**2 #yatak odasının karesini aldık gücünü arttırdık
df['bathrooms']=df['bathrooms']**2 #banyoların karesini aldık gücünü arttırdık ÖNEMİNİ ARTTIRDIK
df['sqft_living']=df['sqft_living']**2 #yaşama alanının karesini aldık
df['grade']=df['grade']**2
df['condition']=df['condition']**2
#df['basement']=np.where(df['sqft_basement']>0,1,0) # bodrum katı varsa 1 yaz
#qdf['renovated']=np.where(df['yr_renovated']>0,1,0) # yenilendiyse 1 yaz

outliers=df.quantile(.97) # 0.97 sağdan %3, 0.03 soldan keser
df=df[df['price']<outliers['price']]
df=df[df['bedrooms']<outliers['bedrooms']]

In [5]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,age
0,7129300520,20141013T000000,221900.0,9,1.0000,1392400,5650,1.0,0,0,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,68
1,6414100192,20141209T000000,538000.0,9,5.0625,6604900,7242,2.0,0,0,...,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,72
2,5631500400,20150225T000000,180000.0,4,1.0000,592900,10000,1.0,0,0,...,770,0,1933,0,98028,47.7379,-122.233,2720,8062,90
3,2487200875,20141209T000000,604000.0,16,9.0000,3841600,5000,1.0,0,0,...,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,58
4,1954400510,20150218T000000,510000.0,9,4.0000,2822400,8080,1.0,0,0,...,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,36


In [221]:
x=df[['bedrooms', 'bathrooms', 'sqft_living', 'grade',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'zipcode', 'sqft_living15', 'sqft_lot15', 'age', 'basement', 'renovated']]
y=df['price']

x=pd.get_dummies(x,drop_first=True) 

x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.20,random_state=42)

lr = LinearRegression()
R=Ridge()
L=Lasso()

## Model 1

In [222]:
model=lr.fit(x_train,y_train) #Fit makinanın öğrenmesini sağlıyor

In [223]:
tahmin=model.predict(x_test) #Tahmin ettik

In [224]:
r2_score(tahmin,y_test) #7983

0.813936529946313

In [225]:
mean_squared_error(tahmin,y_test)**.5 #86049

89133.55866267449

## Model 2

In [226]:
model2=L.fit(x_train,y_train)

In [227]:
tahmin2=model2.predict(x_test)

In [228]:
r2_score(tahmin2,y_test)

0.8138587361746479

In [229]:
mean_squared_error(tahmin2,y_test)**.5

89131.15822160276

## Model 3

In [230]:
model3=L.fit(x_train,y_train)

In [231]:
tahmin3=model3.predict(x_test)

In [232]:
r2_score(tahmin3,y_test)

0.8138587361746479

In [233]:
mean_squared_error(tahmin3,y_test)**.5

89131.15822160276

### HATA HESAPLAMA

In [234]:
mean_squared_error(tahmin,y_test)**.5  #HATAYI HESAPLIYOR LinearRegression()

89133.55866267449

In [235]:
mean_squared_error(tahmin2,y_test)**.5 #HATAYI HESAPLIYOR Lasso()

89131.15822160276

In [236]:
mean_squared_error(tahmin3,y_test)**.5 #HATAYI HESAPLIYOR Ridge()

89131.15822160276

# HEDEF = R2> 0.80

### SKOR

In [237]:
r2_score(tahmin,y_test)

0.813936529946313

In [238]:
r2_score(tahmin2,y_test)

0.8138587361746479

In [239]:
r2_score(tahmin3,y_test)

0.8138587361746479